## Enter Credentials and authenticate login to ArcGIS Online Organization

In [1]:
from arcgis.gis import GIS
import getpass
username = "SDocherty_AFC" #Seth's username
#username = "lwright_AFC" # Lyle's Username
password = getpass.getpass("Enter Password: ")
gis = GIS("https://arcgisforcommer.maps.arcgis.com", username, password)
print("Successfully logged in as: " + gis.properties.user.username)

Enter Password: ········
Successfully logged in as: SDocherty_AFC


## Using the python package, feedparser, let's take an example incident feed and format it into a Feed Parser Dict object for easy manipulation of the dataset


#### Check out the following link for more information on feedparser and GeoRSS
https://sgillies.net/2012/03/06/feedparser-and-georssgml.html

https://pythonhosted.org/feedparser/

In [2]:
import feedparser

# Demo Disease Outbreaks Map Feed GeoRSS feed from GIM
feed = feedparser.parse("http://www.globalincidentmap.com/georss/?7Dt65RFaio90Q0FcHRtjgKwVuo3SpjQ05zN4yvwVQCt_*e447a")

# Print the first line from the feed
print(feed.entries[0])

{'title': 'INDIA - Swine Flu Claims 7 Lives In Chhattisgarh', 'title_detail': {'type': 'text/plain', 'language': None, 'base': 'http://www.globalincidentmap.com/georss/?7Dt65RFaio90Q0FcHRtjgKwVuo3SpjQ05zN4yvwVQCt_*e447a', 'value': 'INDIA - Swine Flu Claims 7 Lives In Chhattisgarh'}, 'summary': 'Swine Flu - Confirmed / Possible Related Death - India,Chhattisgarh:<br><div>&nbsp;<br  />\n<strong>[UNI] INDIA - Swine flu claims 7 lives in Chhattisgarh<br  />\n<br type="_moz"  />\n</strong></div>\n<div><strong>&ldquo;Claiming to have taken adequate steps for prevention of swine flu in Chhattisgarh, Health Minister TS Singhdeo ....&rdquo;<br  />\n<br type="_moz"  />\n</strong></div>\n<div><strong>Read the full article: <br  />\n</strong><a href="http://www.uniindia.com/swine-flu-claims-7-lives-in-chhattisgarh-singhdeo/north/news/1508619.html"><strong>http://www.uniindia.com/swine-flu-claims-7-lives-in-chhattisgarh-singhdeo/north/news/1508619.html</strong></a></div><p>This incident provided vi

In [3]:
# Lets take a look at the title of the first incident:
print(feed.entries[0]["title"])

INDIA - Swine Flu Claims 7 Lives In Chhattisgarh


## What other fields do we have access to?

In [4]:
#print a list of dictionary keys
[item for item in feed.entries[0].keys()]

['title',
 'title_detail',
 'summary',
 'summary_detail',
 'links',
 'link',
 'published',
 'published_parsed',
 'tags',
 'id',
 'guidislink',
 'geo_long',
 'geo_lat',
 'locationgranularity']

In [5]:
sms = {"color": [255,0,0,255], "size": 5, "type": "esriSMS", "style": "esriSMSCircle"}
map1 = gis.map()
record_list = []
for item in range (len(feed.entries)):
    lat = feed.entries[item]["geo_lat"]
    long = feed.entries[item]["geo_long"]
    title = feed.entries[item]["title"]
    content = feed.entries[item]["summary"]
    
    # Prep data for pandas dataframe
    record_list.append([lat, long, title, content])
    
    #Let's draw the content real quick to see what it looks like
    coords = [feed.entries[item]["geo_lat"], feed.entries[item]["geo_long"]]
    map1.draw(coords, symbol=sms, popup={'title':title, 'content':content})
map1 # You can click on the points to expose information about them.

MapView(layout=Layout(height='400px', width='100%'))

## How many records are showing up on the map?

In [6]:
len(record_list)

100

## Let's try the alternative to adding data into the map by creating a feature collection

### With this approach, we can perform spatial analysis with this layer

In [7]:
import pandas as pd
df = pd.DataFrame.from_records(record_list)
df.columns = ['lat', 'long', 'title', 'summary']
incidents = gis.content.import_data(df)
map2 = gis.map()
map2.add_layer(incidents)
map2

MapView(layout=Layout(height='400px', width='100%'))

###### Let us publish this layer as a feature collection item in our GIS

In [8]:
import json
incidents_item_properties = {'title': 'Incidents',
                        'description':'Demo Disease Outbreaks Map Feed GeoRSS feed from GIM',
                        'tags': 'arcgis python api, pandas, GIM',
                        'text': json.dumps({"featureCollection": {"layers": [dict(incidents.layer)]}}),
                        'type':'Feature Collection'}
incidents_item = gis.content.add(incidents_item_properties)
incidents_item

<Item title:"Incidents" type:Feature Collection owner:SDocherty_AFC>